# PySpark 字符串处理

In [1]:
from pyspark.sql.types import *
from pyspark.sql import functions as F

## 基本字符串处理

### 字符串格式拼接字符串

In [2]:
df = spark.createDataFrame([(5, "hello")], ['a', 'b'])
df = df.withColumn('v', F.format_string('%d %s', df.a, df.b))
df.show()

+---+-----+-------+
|  a|    b|      v|
+---+-----+-------+
|  5|hello|5 hello|
+---+-----+-------+



### 字符串位置

In [3]:
df.select(F.instr(df.v, 'h').alias('pos')).show()

+---+
|pos|
+---+
|  3|
+---+



### 字符串截取

In [4]:
df.select(F.substring(df.v, 1, 5).alias('substring')).show()

+---------+
|substring|
+---------+
|    5 hel|
+---------+



## 替换 `DataFrame` 中的列值

- `regexp_replace()`、`translate()`、 `overlay()`来替换 `PySpark DataFrame` 的列值。

### 创建一个带有一些地址的 `PySpark DataFrame`
使用这个 DataFrame 来解释如何替换列值。

In [5]:
address = [
    (1,"14851 Jeffrey Rd","DE"),
    (2,"43421 Margarita St","NY"),
    (3,"13111 Siemon Ave","CA")]
df =spark.createDataFrame(
    address, ["id", "address", "state"])
df.show()

+---+------------------+-----+
| id|           address|state|
+---+------------------+-----+
|  1|  14851 Jeffrey Rd|   DE|
|  2|43421 Margarita St|   NY|
|  3|  13111 Siemon Ave|   CA|
+---+------------------+-----+



### `regexp_replace()`替换字符串列值

使用正则表达式进行匹配，如果正则表达式不匹配则返回空字符串，

下面的示例将字段`address`名称`Rd`值替换为`Road`字符串。

In [6]:
df.withColumn('address', F.regexp_replace('address', 'Rd', 'Road')).show()

+---+------------------+-----+
| id|           address|state|
+---+------------------+-----+
|  1|14851 Jeffrey Road|   DE|
|  2|43421 Margarita St|   NY|
|  3|  13111 Siemon Ave|   CA|
+---+------------------+-----+



### 有条件地替换列值

在上面的例子中，只是替换`Rd`，但没有替换`St`和`Ave`，
使用`when().otherwise()`条件函数有条件地替换列值。

In [7]:
df.withColumn('address', 
    F.when(df.address.endswith('Rd'), F.regexp_replace(df.address,'Rd','Road'))\
   .when(df.address.endswith('St'), F.regexp_replace(df.address,'St','Street'))\
   .when(df.address.endswith('Ave'), F.regexp_replace(df.address,'Ave','Avenue'))\
   .otherwise(df.address)).show(truncate=False)

+---+----------------------+-----+
|id |address               |state|
+---+----------------------+-----+
|1  |14851 Jeffrey Road    |DE   |
|2  |43421 Margarita Street|NY   |
|3  |13111 Siemon Avenue   |CA   |
+---+----------------------+-----+



### 用字典替换列值

在下面的示例中，我们将`state`列的字符串值替换为字典键值对中的完整缩写名称，
为此我使用`PySpark map()`来循环遍历 `DataFrame` 的每一行。

#### 方法一：

In [8]:
stateDic={'CA':'California','NY':'New York','DE':'Delaware'}
df2=df.rdd.map(lambda x: (x.id, x.address, stateDic[x.state]))\
    .toDF(["id", "address", "state"])
df2.show()

+---+------------------+----------+
| id|           address|     state|
+---+------------------+----------+
|  1|  14851 Jeffrey Rd|  Delaware|
|  2|43421 Margarita St|  New York|
|  3|  13111 Siemon Ave|California|
+---+------------------+----------+



#### 方法二：

- 参数：`subset` 指定要替换的列

In [9]:
df.replace(stateDic, subset='state').show()
# 或者
df.replace(list(stateDic.keys()), list(stateDic.values()), subset='state').show()

+---+------------------+----------+
| id|           address|     state|
+---+------------------+----------+
|  1|  14851 Jeffrey Rd|  Delaware|
|  2|43421 Margarita St|  New York|
|  3|  13111 Siemon Ave|California|
+---+------------------+----------+

+---+------------------+----------+
| id|           address|     state|
+---+------------------+----------+
|  1|  14851 Jeffrey Rd|  Delaware|
|  2|43421 Margarita St|  New York|
|  3|  13111 Siemon Ave|California|
+---+------------------+----------+



### 逐个字符替换列值`translate()`

逐个字符地替换 `DataFrame` 列值。在下面的例子中，
每一个字符`1`替换为`A`，`2`替换为`B`，`3`替换为`C`。

In [10]:
df.withColumn('address', F.translate('address', '123', 'ABC')).show()

+---+------------------+-----+
| id|           address|state|
+---+------------------+-----+
|  1|  A485A Jeffrey Rd|   DE|
|  2|4C4BA Margarita St|   NY|
|  3|  ACAAA Siemon Ave|   CA|
+---+------------------+-----+



### 用另一个列值替换列

通过使用`expr()` 和 `regexp_replace()`可以用另一个 `DataFrame column` 中的值替换列值。

In [11]:
df = spark.createDataFrame(
   [("ABCDE_XYZ", "XYZ","FGH")], 
    ("col1", "col2","col3"))
df.withColumn(
    "new_column", F.expr("regexp_replace(col1, col2, col3)").alias("replaced_value")).show()

+---------+----+----+----------+
|     col1|col2|col3|new_column|
+---------+----+----+----------+
|ABCDE_XYZ| XYZ| FGH| ABCDE_FGH|
+---------+----+----+----------+



### 使用`overlay()`函数，用来自另一列的字符串值替换列值。

`overlay()`函数，用于从开始位置和字符数将字符串与另一个列字符串覆盖。

In [12]:
df = spark.createDataFrame([("ABCDE_XYZ", "FGH")], ("col1", "col2"))
df.select('col1', 'col2', F.overlay("col1", "col2", 7).alias("overlayed")).show()

+---------+----+---------+
|     col1|col2|overlayed|
+---------+----+---------+
|ABCDE_XYZ| FGH|ABCDE_FGH|
+---------+----+---------+



## 正则表达式

### 正则表达式提取

In [13]:
df = spark.createDataFrame([('100-200', '中午01')], ['str', 'cn'])
df.select('str',
    F.regexp_extract('str', '(\d+)-(\d+)', 0).alias('pos-0'),
    F.regexp_extract('str', '(\d+)-(\d+)', 1).alias('pos-1'),
    F.regexp_extract('str', '(\d+)-(\d+)', 2).alias('pos-2'),
    F.regexp_extract(df.cn, "[\u4e00-\u9fa5]+", 0).alias('中文'),  # 提取全部的中文字符串
).show()

+-------+-------+-----+-----+----+
|    str|  pos-0|pos-1|pos-2|中文|
+-------+-------+-----+-----+----+
|100-200|100-200|  100|  200|中午|
+-------+-------+-----+-----+----+



### 字符串拼接

In [14]:
df = spark.createDataFrame([('abcd','123')], ['str', 'int'])
df_new = df.select(
    F.concat(df.str, df.int).alias('concat'),    # 直接拼接
    F.concat_ws('-', df.str, df.int).alias('concat_ws'), # 指定拼接符
)
df_new.show()

+-------+---------+
| concat|concat_ws|
+-------+---------+
|abcd123| abcd-123|
+-------+---------+



### 字符串重复`repeat`

In [15]:
# df 中的 str 列重复两次
df.select(F.repeat('str', 2).alias('str 重复两次')).show()

+------------+
|str 重复两次|
+------------+
|    abcdabcd|
+------------+



### 分割`split`

`F.split(str, pattern, limit=-1)`

- `limit <= 0`: 展示全部的拆分内容
- `limit > 0`: 不超过展示的最小条目数

In [16]:
df_new.select(
    'concat_ws',
    F.split('concat_ws', '-', -1).alias('split array'), 
    F.split('concat_ws', '-', 0).alias('0'),
    F.split('concat_ws', '-', 1).alias('1'),
    F.split('concat_ws', '-', 2).alias('2'),
).show()

+---------+-----------+-----------+----------+-----------+
|concat_ws|split array|          0|         1|          2|
+---------+-----------+-----------+----------+-----------+
| abcd-123|[abcd, 123]|[abcd, 123]|[abcd-123]|[abcd, 123]|
+---------+-----------+-----------+----------+-----------+



### `contains` 或 `rlike` 进行行选取

In [17]:
# 展示包含 - 的数据
df_new.where(df_new.concat_ws.contains("-")).show()

# 展示包含 - 或 = 的数据
df_new.where(df_new.concat_ws.rlike("-|=")).show()

+-------+---------+
| concat|concat_ws|
+-------+---------+
|abcd123| abcd-123|
+-------+---------+

+-------+---------+
| concat|concat_ws|
+-------+---------+
|abcd123| abcd-123|
+-------+---------+



### 修改数据类型

In [18]:
# 查看数据类型
df = spark.createDataFrame([('abcd', 123)], ['str', 'int'])
df.printSchema()
df.dtypes

root
 |-- str: string (nullable = true)
 |-- int: long (nullable = true)



[('str', 'string'), ('int', 'bigint')]

#### 方式一：`withColumn` 覆盖原字段

In [19]:
# cast
df.withColumn('int', df.int.cast(DoubleType())).printSchema()

root
 |-- str: string (nullable = true)
 |-- int: double (nullable = true)



In [20]:
# astype
df.withColumn("int", df.int.astype(StringType())).printSchema()

root
 |-- str: string (nullable = true)
 |-- int: string (nullable = true)



#### 方式二：Select

In [21]:
# cast
df.select(F.col('int').cast(StringType())).printSchema()

# astype
df.select(F.col('int').astype(StringType())).printSchema()

root
 |-- int: string (nullable = true)

root
 |-- int: string (nullable = true)



---